This notebook will be mainly used for the capstone project for the IBM Data Science Course

In [189]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4

In [190]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
if page.status_code == 200:
    content = page.content
table_contents=[]
soup = BeautifulSoup(content, 'html.parser')
table=soup.find('table')
for row in table.findAll('td'):
    cell={}
    if row.a != None:
        postal_code=row.b.text
        borough=row.span.a.text
      
        try: neighborhood=row.select('span > a')[1:][0].text
        
        except IndexError:
            
            neighborhood=row.span.a.next_sibling.next_sibling            

            if neighborhood==None:
                neighborhood=borough
                borough=row.span.a.previous_sibling.previous_sibling.previous_sibling
        if type(neighborhood)==bs4.element.NavigableString:
                neighborhood=neighborhood.replace('(','').replace(')','')       
        cell['postal_code']=postal_code
        cell['borough']=borough
        cell['neighborhood']=neighborhood
        table_contents.append(cell)
df=pd.DataFrame(table_contents)    
    
        
            

In [191]:
df.head(20)

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill
9,M5B,Downtown Toronto,Garden District


In [192]:
df.shape

(101, 3)

In [193]:
cords=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

temp_table_content=[]
for row, post_cod in enumerate(df.postal_code):
    cell_2={}
    latitude=cords[cords['Postal Code']==post_cod]['Latitude']
    longitude=cords[cords['Postal Code']==post_cod]['Longitude']
    cell_2['Latitude']=float(latitude)
    cell_2['Longitude']=float(longitude)
    temp_table_content.append(cell_2)
df_temp=pd.DataFrame(temp_table_content)
    
df_total=pd.concat([df,df_temp],axis=1)    

In [194]:
df_total.head(20)

,postal_code,borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill,43.706397,-79.309937
9,M5B,Downtown Toronto,Garden District,43.657162,-79.378937


In [195]:
df_total.shape

(101, 5)

In [196]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium

In [197]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_total['Latitude'], df_total['Longitude'], df_total['borough'], df_total['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
